In [30]:
import pandas as pd
import geopandas as gp
pd.set_option('display.max_columns', 500)

In [2]:
##http://schools.nyc.gov/Accountability/data/GraduationDropoutReports/default.htm

grad = pd.read_csv('graduation_2006_cohort.csv')
grad.head()

,id,school,cohort_year,cohort_category,total_students,#_total_grads,per_total
0,01M056,CORLEARS SCHOOL,2001,4 Year June,1,s,s
1,01M056,CORLEARS SCHOOL,2001,5 Year June,1,s,s
2,01M056,CORLEARS SCHOOL,2001,6 Year,1,s,s
3,01M056,CORLEARS SCHOOL,2003,4 Year June,1,s,s
4,01M056,CORLEARS SCHOOL,2003,5 Year June,1,s,s


In [4]:
#filter the cohort year to 2011
#this class graduated in 2015
grad = grad[grad.cohort_year == 2011]
grad.head()

,id,school,cohort_year,cohort_category,total_students,#_total_grads,per_total
42,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,2011,4 Year June,73,46,63.0%
43,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,2011,4 Year August,73,46,63.0%
84,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,2011,4 Year June,82,68,82.9%
85,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,2011,4 Year August,82,70,85.4%
126,01M450,EAST SIDE COMMUNITY SCHOOL,2011,4 Year June,88,75,85.2%


In [5]:
#filter to only graduates who graduated in 4 years and in june
june_grad = grad[grad.cohort_category == '4 Year  June']
june_grad.head()

,id,school,cohort_year,cohort_category,total_students,#_total_grads,per_total
42,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,2011,4 Year June,73,46,63.0%
84,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,2011,4 Year June,82,68,82.9%
126,01M450,EAST SIDE COMMUNITY SCHOOL,2011,4 Year June,88,75,85.2%
151,01M458,FORSYTH SATELLITE ACADEMY,2011,4 Year June,87,13,14.9%
193,01M509,MARTA VALLE HIGH SCHOOL,2011,4 Year June,102,45,44.1%


In [11]:
june_grad = june_grad.convert_objects(convert_numeric=True)

In [12]:
june_grad.dtypes

id                  object
school              object
cohort_year          int64
cohort_category     object
total_students     float64
#_total_grads      float64
per_total           object
dtype: object

In [13]:
#create new column to get percentage of graduates at each school
june_grad['grad_per'] = june_grad['#_total_grads']/june_grad['total_students']
june_grad.head()

,id,school,cohort_year,cohort_category,total_students,#_total_grads,per_total,grad_per
42,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,2011,4 Year June,73,46,63.0%,0.630137
84,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,2011,4 Year June,82,68,82.9%,0.829268
126,01M450,EAST SIDE COMMUNITY SCHOOL,2011,4 Year June,88,75,85.2%,0.852273
151,01M458,FORSYTH SATELLITE ACADEMY,2011,4 Year June,87,13,14.9%,0.149425
193,01M509,MARTA VALLE HIGH SCHOOL,2011,4 Year June,102,45,44.1%,0.441176


In [6]:
#import file with schools and the school district number to merge with the june_grad file. 
school_lookup = pd.read_csv('schools_to_zones_lookup.csv')
school_lookup.head()

,ats_code,loc_code,schoolname,sch_type,geo_district,admin_dist,address,zip,grades,lon,lat,esid_no,msid_no,hsid_no
0,06M005,M005,P.S. 005 ELLEN LURIE,Elementary,6,6,3703 TENTH AVENUE,10034,"0K,01,02,03,04,05,SE",-73.922359,40.858479,589,105,4
1,01M364,M364,EARTH SCHOOL,Elementary,1,1,600 EAST 6 STREET,10009,"PK,0K,01,02,03,04,05,SE",-73.981291,40.724087,502,81,4
2,09X011,X011,P.S. 011 HIGHBRIDGE,Elementary,9,9,1257 OGDEN AVENUE,10452,"0K,01,02,03,04,05,SE",-73.925783,40.839693,777,155,9
3,32K106,K106,P.S. 106 EDWARD EVERETT HALE,Elementary,32,32,1328 PUTNAM AVENUE,11221,"PK,0K,01,02,03,04,05,SE",-73.912672,40.693700,88,406,42
4,10X546,X546,BRONX THEATRE HIGH SCHOOL,High school,10,10,99 TERRACE VIEW AVENUE,10463,"09,10,11,12,SE",-73.912953,40.877868,830,340,71


In [8]:
school_lookup.dtypes

ats_code         object
loc_code         object
schoolname       object
sch_type         object
geo_district      int64
admin_dist        int64
address          object
zip               int64
grades           object
lon             float64
lat             float64
esid_no           int64
msid_no           int64
hsid_no           int64
dtype: object

In [9]:
#change column name 'ats_code' to 'id' to be able to merge with june_grad file 
school_lookup = school_lookup.rename(columns={'ats_code': 'id'})
school_lookup.head()

,id,loc_code,schoolname,sch_type,geo_district,admin_dist,address,zip,grades,lon,lat,esid_no,msid_no,hsid_no
0,06M005,M005,P.S. 005 ELLEN LURIE,Elementary,6,6,3703 TENTH AVENUE,10034,"0K,01,02,03,04,05,SE",-73.922359,40.858479,589,105,4
1,01M364,M364,EARTH SCHOOL,Elementary,1,1,600 EAST 6 STREET,10009,"PK,0K,01,02,03,04,05,SE",-73.981291,40.724087,502,81,4
2,09X011,X011,P.S. 011 HIGHBRIDGE,Elementary,9,9,1257 OGDEN AVENUE,10452,"0K,01,02,03,04,05,SE",-73.925783,40.839693,777,155,9
3,32K106,K106,P.S. 106 EDWARD EVERETT HALE,Elementary,32,32,1328 PUTNAM AVENUE,11221,"PK,0K,01,02,03,04,05,SE",-73.912672,40.693700,88,406,42
4,10X546,X546,BRONX THEATRE HIGH SCHOOL,High school,10,10,99 TERRACE VIEW AVENUE,10463,"09,10,11,12,SE",-73.912953,40.877868,830,340,71


In [14]:
#merge the files to get district number for each school
school_grad = pd.merge(school_lookup, june_grad, on='id')
school_grad.head()

,id,loc_code,schoolname,sch_type,geo_district,admin_dist,address,zip,grades,lon,...,esid_no,msid_no,hsid_no,school,cohort_year,cohort_category,total_students,#_total_grads,per_total,grad_per
0,10X546,X546,BRONX THEATRE HIGH SCHOOL,High school,10,10,99 TERRACE VIEW AVENUE,10463,"09,10,11,12,SE",-73.912953,...,830,340,71,BRONX THEATRE HIGH SCHOOL,2011,4 Year June,102,63,61.8%,0.617647
1,28Q325,Q325,HILLSIDE ARTS & LETTERS ACADEMY,High school,28,28,167-01 GOTHIC DRIVE,11432,"09,SE",-73.798106,...,355,319,56,HILLSIDE ARTS & LETTERS ACADEMY,2011,4 Year June,100,76,76.0%,0.760000
2,13K265,K265,DR. SUSAN S. MCKINNEY SECONDARY SCHOOL OF THE ...,Secondary School,13,13,101 PARK AVENUE,11205,"06,07,08,09,10,11,12,SE",-73.975948,...,248,291,42,DR. SUSAN S. MCKINNEY SECONDARY SCHOOL OF THE ...,2011,4 Year June,69,34,49.3%,0.492754
3,14K322,K322,FOUNDATIONS ACADEMY,High school,14,14,70 TOMPKINS AVENUE,11206,"09,10,11,12,SE",-73.946852,...,29,304,42,FOUNDATIONS ACADEMY,2011,4 Year June,36,12,33.3%,0.333333
4,21K344,K344,RACHEL CARSON HIGH SCHOOL FOR COASTAL STUDIES,High school,21,21,521 WEST AVENUE,11224,"09,10,11,12,SE",-73.971936,...,124,183,23,RACHEL CARSON HIGH SCHOOL FOR COASTAL STUDIES,2011,4 Year June,150,121,80.7%,0.806667


In [18]:
#grouping by district number and cohort year
total_grad = school_grad.groupby(['geo_district','cohort_year']).sum()

In [20]:
#checking to make sure the grouping worked
total_grad

,,admin_dist,zip,lon,lat,esid_no,msid_no,hsid_no,total_students,#_total_grads,grad_per
geo_district,cohort_year,,,,,,,,,,
1,2011,9,90025,-665.861938,366.477236,4518,729,676,1081,648,5.502511
2,2011,136,671540,-4957.318526,2729.688700,38226,6122,1546,8644,6213,44.647054
3,2011,45,150357,-1109.659554,611.741354,9537,5109,60,2374,1766,9.806302
4,2011,24,60174,-443.662012,244.755282,3355,582,24,707,633,4.419719
5,2011,50,100311,-739.470436,408.152546,5728,1298,40,932,616,6.366440
6,2011,60,100363,-739.312634,408.504853,5889,1641,40,1181,737,6.039622
7,2011,112,167243,-1182.708588,653.066514,12518,3504,144,1397,771,8.466028
8,2011,136,177958,-1255.733266,693.981226,12595,2422,159,1952,895,8.248657
9,2011,198,230035,-1625.938432,898.397816,14715,3367,198,2113,1380,13.724788


In [21]:
#calculating new percent of graduates per school district 
total_grad['percent_grads'] = total_grad['#_total_grads']/total_grad['total_students']
total_grad.head()

,,admin_dist,zip,lon,lat,esid_no,msid_no,hsid_no,total_students,#_total_grads,grad_per,percent_grads
geo_district,cohort_year,,,,,,,,,,,
1,2011,9,90025,-665.861938,366.477236,4518,729,676,1081,648,5.502511,0.599445
2,2011,136,671540,-4957.318526,2729.688700,38226,6122,1546,8644,6213,44.647054,0.718764
3,2011,45,150357,-1109.659554,611.741354,9537,5109,60,2374,1766,9.806302,0.743892
4,2011,24,60174,-443.662012,244.755282,3355,582,24,707,633,4.419719,0.895332
5,2011,50,100311,-739.470436,408.152546,5728,1298,40,932,616,6.366440,0.660944


In [22]:
#geo district is school district
total_grad = total_grad.reset_index(drop=False)
total_grad.head()

,geo_district,cohort_year,admin_dist,zip,lon,lat,esid_no,msid_no,hsid_no,total_students,#_total_grads,grad_per,percent_grads
0,1,2011,9,90025,-665.861938,366.477236,4518,729,676,1081,648,5.502511,0.599445
1,2,2011,136,671540,-4957.318526,2729.688700,38226,6122,1546,8644,6213,44.647054,0.718764
2,3,2011,45,150357,-1109.659554,611.741354,9537,5109,60,2374,1766,9.806302,0.743892
3,4,2011,24,60174,-443.662012,244.755282,3355,582,24,707,633,4.419719,0.895332
4,5,2011,50,100311,-739.470436,408.152546,5728,1298,40,932,616,6.366440,0.660944


In [23]:
#export to csv
#total_grad.to_csv('total_grad.csv')

In [24]:
#http://schools.nyc.gov/AboutUs/schools/data/default.htm
#demographic snapshots link
schools = pd.read_csv('schools.csv')
schools_df = schools[schools.Year=='2015-16']
schools_df.head()

,DBN,School Name,Year,Total Enrollment,Grade PK,Grade K,Grade 1,Grade 2,Grade 3,Grade 4,...,# Other,% Other,# White,% White,# Students with Disabilities,% Students with Disabilities,# English Language Learners,% English Language Learners,# Poverty,% Poverty
4,01M015,P.S. 015 Roberto Clemente,2015-16,176,14,32,33,39,23,17,...,3,1.70%,2,1.10%,57,32.40%,16,9.10%,176,100.00%
9,01M019,P.S. 019 Asher Levy,2015-16,270,21,47,43,41,43,35,...,3,1.10%,16,5.90%,80,29.60%,13,4.80%,217,80.40%
14,01M020,P.S. 020 Anna Silver,2015-16,581,46,82,90,98,92,81,...,21,3.60%,24,4.10%,122,21.00%,88,15.10%,581,100.00%
19,01M034,P.S. 034 Franklin D. Roosevelt,2015-16,394,18,33,41,30,27,36,...,0,0.00%,14,3.60%,114,28.90%,29,7.40%,383,97.20%
24,01M063,The STAR Academy - P.S.63,2015-16,203,15,47,42,35,28,19,...,2,1.00%,21,10.30%,64,31.50%,6,3.00%,171,84.20%


In [ ]:
schools_df = schools_df.convert_objects(convert_numeric=True)

In [26]:
schools_df.replace('%', '', regex=True, inplace=True)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [28]:
schools_df.head()

,DBN,School Name,Year,Total Enrollment,Grade PK,Grade K,Grade 1,Grade 2,Grade 3,Grade 4,...,# Other,% Other,# White,% White,# Students with Disabilities,% Students with Disabilities,# English Language Learners,% English Language Learners,# Poverty,% Poverty
4,01M015,P.S. 015 Roberto Clemente,2015-16,176,14,32,33,39,23,17,...,3,1.70,2,1.10,57,32.40,16,9.10,176,100.00
9,01M019,P.S. 019 Asher Levy,2015-16,270,21,47,43,41,43,35,...,3,1.10,16,5.90,80,29.60,13,4.80,217,80.40
14,01M020,P.S. 020 Anna Silver,2015-16,581,46,82,90,98,92,81,...,21,3.60,24,4.10,122,21.00,88,15.10,581,100.00
19,01M034,P.S. 034 Franklin D. Roosevelt,2015-16,394,18,33,41,30,27,36,...,0,0.00,14,3.60,114,28.90,29,7.40,383,97.20
24,01M063,The STAR Academy - P.S.63,2015-16,203,15,47,42,35,28,19,...,2,1.00,21,10.30,64,31.50,6,3.00,171,84.20


In [29]:
schools_df['% Poverty'] = schools_df['% Poverty'].astype(float)/100
schools_df['% English Language Learners'] = schools_df['% English Language Learners'].astype(float)/100
schools_df['% Students with Disabilities'] = schools_df['% Students with Disabilities'].astype(float)/100
schools_df['% White'] = schools_df['% White'].astype(float)/100
schools_df['% Other'] = schools_df['% Other'].astype(float)/100
schools_df['% Hispanic'] = schools_df['% Hispanic'].astype(float)/100
schools_df['% Black'] = schools_df['% Black'].astype(float)/100
schools_df['% Asian'] = schools_df['% Asian'].astype(float)/100
schools_df['% Male'] = schools_df['% Male'].astype(float)/100
schools_df['% Female'] = schools_df['% Female'].astype(float)/100
schools_df.head()


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: 

,DBN,School Name,Year,Total Enrollment,Grade PK,Grade K,Grade 1,Grade 2,Grade 3,Grade 4,...,# Other,% Other,# White,% White,# Students with Disabilities,% Students with Disabilities,# English Language Learners,% English Language Learners,# Poverty,% Poverty
4,01M015,P.S. 015 Roberto Clemente,2015-16,176,14,32,33,39,23,17,...,3,0.017,2,0.011,57,0.324,16,0.091,176,1.000
9,01M019,P.S. 019 Asher Levy,2015-16,270,21,47,43,41,43,35,...,3,0.011,16,0.059,80,0.296,13,0.048,217,0.804
14,01M020,P.S. 020 Anna Silver,2015-16,581,46,82,90,98,92,81,...,21,0.036,24,0.041,122,0.210,88,0.151,581,1.000
19,01M034,P.S. 034 Franklin D. Roosevelt,2015-16,394,18,33,41,30,27,36,...,0,0.000,14,0.036,114,0.289,29,0.074,383,0.972
24,01M063,The STAR Academy - P.S.63,2015-16,203,15,47,42,35,28,19,...,2,0.010,21,0.103,64,0.315,6,0.030,171,0.842


In [31]:
schools_df.head()

,DBN,School Name,Year,Total Enrollment,Grade PK,Grade K,Grade 1,Grade 2,Grade 3,Grade 4,Grade 5,Grade 6,Grade 7,Grade 8,Grade 9,Grade 10,Grade 11,Grade 12,# Female,% Female,# Male,% Male,# Asian,% Asian,# Black,% Black,# Hispanic,% Hispanic,# Other,% Other,# White,% White,# Students with Disabilities,% Students with Disabilities,# English Language Learners,% English Language Learners,# Poverty,% Poverty
4,01M015,P.S. 015 Roberto Clemente,2015-16,176,14,32,33,39,23,17,18,0,0,0,0,0,0,0,83,0.472,93,0.528,9,0.051,57,0.324,105,0.597,3,0.017,2,0.011,57,0.324,16,0.091,176,1.000
9,01M019,P.S. 019 Asher Levy,2015-16,270,21,47,43,41,43,35,40,0,0,0,0,0,0,0,125,0.463,145,0.537,27,0.100,55,0.204,169,0.626,3,0.011,16,0.059,80,0.296,13,0.048,217,0.804
14,01M020,P.S. 020 Anna Silver,2015-16,581,46,82,90,98,92,81,92,0,0,0,0,0,0,0,277,0.477,304,0.523,204,0.351,46,0.079,286,0.492,21,0.036,24,0.041,122,0.210,88,0.151,581,1.000
19,01M034,P.S. 034 Franklin D. Roosevelt,2015-16,394,18,33,41,30,27,36,33,64,62,50,0,0,0,0,200,0.508,194,0.492,18,0.046,113,0.287,249,0.632,0,0.000,14,0.036,114,0.289,29,0.074,383,0.972
24,01M063,The STAR Academy - P.S.63,2015-16,203,15,47,42,35,28,19,17,0,0,0,0,0,0,0,100,0.493,103,0.507,9,0.044,40,0.197,131,0.645,2,0.010,21,0.103,64,0.315,6,0.030,171,0.842


In [33]:
#change DBN column header to 'id' in order to merge with school_lookup file
schools_df = schools_df.rename(columns={'DBN':'id'})
schools_df.head()

,id,School Name,Year,Total Enrollment,Grade PK,Grade K,Grade 1,Grade 2,Grade 3,Grade 4,Grade 5,Grade 6,Grade 7,Grade 8,Grade 9,Grade 10,Grade 11,Grade 12,# Female,% Female,# Male,% Male,# Asian,% Asian,# Black,% Black,# Hispanic,% Hispanic,# Other,% Other,# White,% White,# Students with Disabilities,% Students with Disabilities,# English Language Learners,% English Language Learners,# Poverty,% Poverty
4,01M015,P.S. 015 Roberto Clemente,2015-16,176,14,32,33,39,23,17,18,0,0,0,0,0,0,0,83,0.472,93,0.528,9,0.051,57,0.324,105,0.597,3,0.017,2,0.011,57,0.324,16,0.091,176,1.000
9,01M019,P.S. 019 Asher Levy,2015-16,270,21,47,43,41,43,35,40,0,0,0,0,0,0,0,125,0.463,145,0.537,27,0.100,55,0.204,169,0.626,3,0.011,16,0.059,80,0.296,13,0.048,217,0.804
14,01M020,P.S. 020 Anna Silver,2015-16,581,46,82,90,98,92,81,92,0,0,0,0,0,0,0,277,0.477,304,0.523,204,0.351,46,0.079,286,0.492,21,0.036,24,0.041,122,0.210,88,0.151,581,1.000
19,01M034,P.S. 034 Franklin D. Roosevelt,2015-16,394,18,33,41,30,27,36,33,64,62,50,0,0,0,0,200,0.508,194,0.492,18,0.046,113,0.287,249,0.632,0,0.000,14,0.036,114,0.289,29,0.074,383,0.972
24,01M063,The STAR Academy - P.S.63,2015-16,203,15,47,42,35,28,19,17,0,0,0,0,0,0,0,100,0.493,103,0.507,9,0.044,40,0.197,131,0.645,2,0.010,21,0.103,64,0.315,6,0.030,171,0.842


In [34]:
#merge with school lookup file in order to get school district number 
school_demo_stats = pd.merge(schools_df, school_lookup, on='id')
school_demo_stats.head()

,id,School Name,Year,Total Enrollment,Grade PK,Grade K,Grade 1,Grade 2,Grade 3,Grade 4,Grade 5,Grade 6,Grade 7,Grade 8,Grade 9,Grade 10,Grade 11,Grade 12,# Female,% Female,# Male,% Male,# Asian,% Asian,# Black,% Black,# Hispanic,% Hispanic,# Other,% Other,# White,% White,# Students with Disabilities,% Students with Disabilities,# English Language Learners,% English Language Learners,# Poverty,% Poverty,loc_code,schoolname,sch_type,geo_district,admin_dist,address,zip,grades,lon,lat,esid_no,msid_no,hsid_no
0,01M015,P.S. 015 Roberto Clemente,2015-16,176,14,32,33,39,23,17,18,0,0,0,0,0,0,0,83,0.472,93,0.528,9,0.051,57,0.324,105,0.597,3,0.017,2,0.011,57,0.324,16,0.091,176,1.000,M015,P.S. 015 ROBERTO CLEMENTE,Elementary,1,1,333 EAST 4 STREET,10009,"PK,0K,01,02,03,04,05,SE",-73.978811,40.721891,502,81,4
1,01M019,P.S. 019 Asher Levy,2015-16,270,21,47,43,41,43,35,40,0,0,0,0,0,0,0,125,0.463,145,0.537,27,0.100,55,0.204,169,0.626,3,0.011,16,0.059,80,0.296,13,0.048,217,0.804,M019,P.S. 019 ASHER LEVY,Elementary,1,1,185 1 AVENUE,10003,"PK,0K,01,02,03,04,05,SE",-73.984180,40.729740,502,81,4
2,01M020,P.S. 020 Anna Silver,2015-16,581,46,82,90,98,92,81,92,0,0,0,0,0,0,0,277,0.477,304,0.523,204,0.351,46,0.079,286,0.492,21,0.036,24,0.041,122,0.210,88,0.151,581,1.000,M020,P.S. 020 ANNA SILVER,Elementary,1,1,166 ESSEX STREET,10002,"PK,0K,01,02,03,04,05,SE",-73.986381,40.721275,502,81,84
3,01M034,P.S. 034 Franklin D. Roosevelt,2015-16,394,18,33,41,30,27,36,33,64,62,50,0,0,0,0,200,0.508,194,0.492,18,0.046,113,0.287,249,0.632,0,0.000,14,0.036,114,0.289,29,0.074,383,0.972,M034,P.S. 034 FRANKLIN D. ROOSEVELT,K-8,1,1,730 EAST 12 STREET,10009,"PK,0K,01,02,03,04,05,06,07,08,SE",-73.975221,40.726266,502,81,4
4,01M063,The STAR Academy - P.S.63,2015-16,203,15,47,42,35,28,19,17,0,0,0,0,0,0,0,100,0.493,103,0.507,9,0.044,40,0.197,131,0.645,2,0.010,21,0.103,64,0.315,6,0.030,171,0.842,M063,P.S. 063 WILLIAM MCKINLEY,Elementary,1,1,121 EAST 3 STREET,10009,"PK,0K,01,02,03,04,05,SE",-73.986340,40.724436,502,81,4


In [35]:
len(school_demo_stats)

1598

In [ ]:
#export to csv
#school_demo_stats.to_csv('school_demo_stats.csv')